# Militares em cargos comissionados

Esse notebook tem por objetivo fazer um estudo dos __servidores públicos militares brasileiros__ que estão atuando em atividades civis através de __cargos comissionados__. Todos os dados aqui presentes são dados públicos disponíveis no site do [PORTAL DA TRANSPARÊNCIA](http://www.portaltransparencia.gov.br/).

# Bibliotecas

In [ ]:
%run libs.py

# Ingestão de dados

## dataframe militares

http://www.portaltransparencia.gov.br/download-de-dados/servidores

In [ ]:
def create_dataframe(folder, csv_file_name, df_ind_name,sep='_'):
    
    # listar todos os arquivos zip no diretório selecionado
    files = os.listdir(folder)
    
    # lista que armazenará todos os dataframes
    dfs_list=[]
    
    # Loop para cada arquivo encontrado no diretório selecionado
    for zip_files in files:

        # este módulo fornece ferramentas para criar, ler, escrever, anexar e listar um arquivo ZIP    
        zf = zipfile.ZipFile(folder+'/'+zip_files)

        # cria str com o nome do arquivo csv
        csv_file = (zip_files[0:6] + '_' + csv_file_name)
        
        # lê arquivo csv dentro do zip e transforma em dataframe.
        globals()[df_ind_name + zip_files[0:6]] = pd.read_csv(zf.open(csv_file), encoding='latin-1', sep=';')

        # adiciona uma coluna que conterá informação do nome do arquivo
        globals()[df_ind_name + zip_files[0:6]]['file'] = zip_files[0:6]

        # lista que armazenará todos os dataframes
        dfs_list.append(globals()[df_ind_name + zip_files[0:6]])
        
        # concatenar lista de de dataframes
        df_final = pd.concat(dfs_list)
        
        return df_final

In [ ]:
df_militares = create_dataframe('data/servidores_militares', 'CadastroMilitares.csv', 'df_mil')
df_militares.sample(3)

In [ ]:
# Temos um DataFrame com o seguinte shape
df_militares.shape

In [ ]:
# Save to a compress format
df_militares.to_csv('data/df_militares.csv.gz', compression='gzip')

In [ ]:
# os arquivos originais não serão mais necessários, podemos deletá-los
path = 'data/servidores_militares_'
shutil.rmtree(path)

## dataframe DAS (Direção e Assessoramento Superiores)

http://repositorio.dados.gov.br/dataset/direcao-e-assessoramento-superiores-das

Arquivos estão disponíveis em format `.CSV`. Para isso iremos juntar todos em apenas 1 único DF. Como temos diversos problemas nos arquivos, iremos criar um DF de apenas uma coluna para ajustar a formatação e depois separar em diversas colunas.

In [ ]:
def create_dataframe(folder):
    # listar todos os arquivos no diretório selecionado
    files = os.listdir(folder)

    # lista que armazenará todos os dataframes
    dfs_list=[]
    
    # leitura dinâmica de files
    for file in files:
        
        # ler CSV eadicionar a uma lista
        dfs_list.append(pd.read_csv((folder+'/'+file),encoding='latin-1',sep='separator',header=None,engine='python'))#
    
    # concatenar todos os DFs dentro da lista
    return pd.concat(dfs_list)

In [ ]:
# DataFrame contém diversos problemas que precisam ser ajustados
df_DAS_to_fix = create_dataframe('data/cargodas')

In [ ]:
# colunas para corrigir
col_to_fix = df_DAS_to_fix.columns[0]

### Remover Caracteres que causam problemas na criação de novas colunas

In [ ]:
df_DAS_to_fix[col_to_fix] = df_DAS_to_fix[col_to_fix].str.replace('; BS', ', BS',regex=True)
df_DAS_to_fix[col_to_fix] = df_DAS_to_fix[col_to_fix].str.replace('; D.O.U.', ', D.O.U.',regex=True)
df_DAS_to_fix[col_to_fix] = df_DAS_to_fix[col_to_fix].str.replace('; DOU .*; .*\/.*\/',', DOU .*, .*\/.*\/',regex=True)
df_DAS_to_fix[col_to_fix] = df_DAS_to_fix[col_to_fix].str.replace('; DOU', ', DOU',regex=True)
df_DAS_to_fix[col_to_fix] = df_DAS_to_fix[col_to_fix].str.replace('; NO DOU', ', NO DOU',regex=True)
df_DAS_to_fix[col_to_fix] = df_DAS_to_fix[col_to_fix].str.replace('; PUBLICADA NO DOU', ', PUBLICADA NO DOU',regex=True)
df_DAS_to_fix[col_to_fix] = df_DAS_to_fix[col_to_fix].str.replace(';05/02/2018', ',05/02/2018',regex=True)
df_DAS_to_fix[col_to_fix] = df_DAS_to_fix[col_to_fix].str.replace(';        ;', ';',regex=True)

In [ ]:
# Separar coluna única em diversas colunas
df_DAS = df_DAS_to_fix[col_to_fix].str.split(';', expand = True)

# Usar a primeira linha como coluna 
df_DAS.columns = df_DAS.iloc[0]

# Como usamos header=None, precisamos remover todas as linhas do header original
df_DAS = df_DAS[~df_DAS['Valor da DAS'].str.contains("DAS")]

# Mas esse DF tem diversos espaços antes e após o valor. Vamos excluir.
df_DAS = df_DAS.apply(lambda x: x.str.strip())

# Resultado
df_DAS.head()

In [ ]:
# salvar para um único arquivo 
df_DAS.to_csv('data/cargodas.csv.gz', compression='gzip')

In [ ]:
# os arquivos originais não serão mais necessários, podemos deletá-los
path = 'data/cargodas'
shutil.rmtree(path)